In [4]:
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np
import scipy.stats as stats
import glob
import os

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

from optimization_c_r_rc_100_100_dataset.aco import ACO
from optimization_c_r_rc_100_100_dataset.acs import ACS
from optimization_c_r_rc_100_100_dataset.bso import BSO
from optimization_c_r_rc_100_100_dataset.dka import DKA
from optimization_c_r_rc_100_100_dataset.ga import GA
from optimization_c_r_rc_100_100_dataset.pso import PSO
from optimization_c_r_rc_100_100_dataset.sa import SA
from optimization_c_r_rc_100_100_dataset.tabu_search import TS
from optimization_c_r_rc_100_100_dataset.woa import WOA

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_analysis(model,depot,nodes,num_vehicle):
    start = time.time()
    model.set_model(nodes = nodes,depot = depot, num_vehicle = num_vehicle)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    results = {"solution":[[i["id"] for i in sol] for sol in solution],
              "fitness":fitness,
              "run_time":run_time}
    
    return results

In [5]:
dir_path = "raw_data/c_r_rc_100_100"
files = glob.glob(os.path.join(dir_path,"*.txt"))

In [13]:
def get_data(filename):
    print(filename)
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    #find the start of the line
    start = len([line for line in lines if len(line.strip().split())<9])
    
    parts = list(map(float,lines[start].strip().split()))
    depot = {
        "id": parts[0], #id
        "x": parts[1], #x coordinate
        "y": parts[2], #y coordinate
        "d": parts[3], #service duration or visiting time
        "S": parts[4], #profit of the location
        "f": parts[5], #not relevant for TOPTW
        "a": parts[6], #not relevant for TOPTW
        "list": parts[7], #not relevant for TOPTW
        "O": 0, #opening of time window (earliest time for start of service) for depot its a starting point
        "C": parts[8], #closing of time window (latest time for start of service) for depot its the maximum duration (Tmax)
    }

    nodes = []
    
    for line in lines[start+1:]:
        parts = list(map(float, line.strip().split()))
        if len(parts) >= 9:
            node = {
                "id": parts[0], #id
                "x": parts[1], #x coordinate
                "y": parts[2], #y coordinate
                "d": parts[3], #service duration or visiting time
                "S": parts[4], #profit of the location
                "f": parts[5], #not relevant for TOPTW
                "a": parts[6], #not relevant for TOPTW
                "list": parts[7], #not relevant for TOPTW
                "O": parts[8], #opening of time window (earliest time for start of service)
                "C": parts[9], #closing of time window (latest time for start of service)
            }
            nodes.append(node)
        else:
            continue
    return depot,nodes

In [14]:
df_results = pd.DataFrame()

#scenario
num_vehicle = 3
random_state = 100

for file in files:
    depot,nodes = get_data(file)
    
    results = {
        "method" : [],
        "fitness" : [],
        "run_time" : []
    }
    
    aco = ACO(random_state = random_state)
    results_aco = get_analysis(aco,depot,nodes,num_vehicle)
    results["method"].append("aco")
    
    acs = ACS(random_state = random_state)
    results_acs = get_analysis(acs,depot,nodes,num_vehicle)
    results["method"].append("acs")
    
    bso = BSO(random_state = random_state)
    results_bso = get_analysis(bso,depot,nodes,num_vehicle)
    results["method"].append("bso")
    
    dka = DKA(random_state = random_state)
    results_dka = get_analysis(dka,depot,nodes,num_vehicle)
    results["method"].append("dka")
    
    ga = GA(random_state = random_state)
    results_ga = get_analysis(ga,depot,nodes,num_vehicle)
    results["method"].append("ga")
    
    pso = PSO(random_state = random_state)
    results_pso = get_analysis(pso,depot,nodes,num_vehicle)
    results["method"].append("pso")
    
    sa = SA(random_state = random_state)
    results_sa = get_analysis(sa,depot,nodes,num_vehicle)
    results["method"].append("sa")
    
    ts = TS(random_state = random_state)
    results_ts = get_analysis(ts,depot,nodes,num_vehicle)
    results["method"].append("ts")
    
    woa = WOA(random_state = random_state)
    results_woa = get_analysis(woa,depot,nodes,num_vehicle)
    results["method"].append("woa")
    
    for i in [results_aco,results_acs,results_bso,results_dka,results_ga,results_pso,results_sa,results_ts,results_woa]:
        results["fitness"].append(i["fitness"])
        results["run_time"].append(i["run_time"])
    
    results = pd.DataFrame(results)
    results["dataset"] = file.split("\\")[-1]
    
    df_results = pd.concat([df_results,results])
    
    print("========================================")

raw_data/c_r_rc_100_100\c101.txt


In [21]:
df_results

,method,fitness,run_time,dataset
0,aco,570.0,52.075427,raw_data/c_r_rc_100_100\c101.txt
1,acs,320.0,22.561275,raw_data/c_r_rc_100_100\c101.txt
2,bso,260.0,6.550626,raw_data/c_r_rc_100_100\c101.txt
3,dka,340.0,2.846139,raw_data/c_r_rc_100_100\c101.txt
4,ga,360.0,9.723127,raw_data/c_r_rc_100_100\c101.txt
5,pso,520.0,4.339589,raw_data/c_r_rc_100_100\c101.txt
6,sa,540.0,8.740301,raw_data/c_r_rc_100_100\c101.txt
7,ts,490.0,24.730015,raw_data/c_r_rc_100_100\c101.txt
8,woa,410.0,9.019006,raw_data/c_r_rc_100_100\c101.txt


In [11]:
ga = GA(random_state=100)
results_ga = get_analysis(ga,depot,nodes,num_vehicle)

In [12]:
results_ga

{'solution': [[3.0, 54.0, 16.0, 36.0, 75.0, 47.0],
  [25.0, 97.0, 59.0, 22.0, 69.0],
  [81.0, 7.0, 61.0, 12.0, 91.0, 21.0, 49.0]],
 'fitness': 360.0,
 'run_time': 6.153579950332642}

In [165]:
acs = ACS(random_state=100)
acs.set_model(nodes=nodes,depot=depot,
                num_vehicle=3)
solution,fitness = acs.construct_solution()

In [134]:
aco = ACO(random_state=100)
aco.set_model(nodes=nodes,depot=depot,
                num_vehicle=3)
solution,fitness = aco.construct_solution()

In [138]:
woa = WOA(random_state=100)
woa.set_model(nodes=nodes,depot=depot,
                num_vehicle=3)
solution,fitness = woa.construct_solution()

In [148]:
sa = SA(random_state=100)
sa.set_model(nodes=nodes,depot=depot,
                num_vehicle=3)
solution,fitness = sa.construct_solution()

In [156]:
ts = TS(random_state=100)
ts.set_model(nodes=nodes,depot=depot,
                num_vehicle=3)
solution,fitness = ts.construct_solution()

In [160]:
pso = PSO(random_state=100)
pso.set_model(nodes=nodes,depot=depot,
                num_vehicle=3)
solution,fitness = pso.construct_solution()

In [179]:
bso = BSO(random_state=100)
bso.set_model(nodes=nodes,depot=depot,
                num_vehicle=3)
solution,fitness = bso.construct_solution()

In [189]:
dka = DKA(random_state=100)
dka.set_model(nodes=nodes,depot=depot,
                num_vehicle=3)
solution,fitness = dka.construct_solution()

In [190]:
solution

[[{'id': 19.0,
   'x': 42.0,
   'y': 10.0,
   'd': 10.0,
   'S': 40.0,
   'f': 1.0,
   'a': 1.0,
   'list': 1.0,
   'O': 40.0,
   'C': 148.0},
  {'id': 59.0,
   'x': 10.0,
   'y': 20.0,
   'd': 10.0,
   'S': 19.0,
   'f': 1.0,
   'a': 1.0,
   'list': 1.0,
   'O': 42.0,
   'C': 187.0},
  {'id': 57.0,
   'x': 30.0,
   'y': 25.0,
   'd': 10.0,
   'S': 23.0,
   'f': 1.0,
   'a': 1.0,
   'list': 1.0,
   'O': 56.0,
   'C': 165.0},
  {'id': 23.0,
   'x': 38.0,
   'y': 5.0,
   'd': 10.0,
   'S': 30.0,
   'f': 1.0,
   'a': 1.0,
   'list': 1.0,
   'O': 45.0,
   'C': 164.0},
  {'id': 91.0,
   'x': 49.0,
   'y': 42.0,
   'd': 10.0,
   'S': 13.0,
   'f': 1.0,
   'a': 1.0,
   'list': 1.0,
   'O': 89.0,
   'C': 217.0},
  {'id': 80.0,
   'x': 47.0,
   'y': 47.0,
   'd': 10.0,
   'S': 13.0,
   'f': 1.0,
   'a': 1.0,
   'list': 1.0,
   'O': 93.0,
   'C': 222.0}],
 [{'id': 51.0,
   'x': 55.0,
   'y': 20.0,
   'd': 10.0,
   'S': 19.0,
   'f': 1.0,
   'a': 1.0,
   'list': 1.0,
   'O': 45.0,
   'C': 150.0},

In [191]:
fitness

358.0

In [113]:
depot

{'id': 0.0,
 'x': 40.0,
 'y': 50.0,
 'd': 0.0,
 'S': 0.0,
 'f': 0.0,
 'a': 0.0,
 'list': 0.0,
 'O': 0,
 'C': 240.0}

In [136]:
np.array([{"a":1,"b":2},{"a":1,"b":2}])

array([{'a': 1, 'b': 2}, {'a': 1, 'b': 2}], dtype=object)

In [141]:
a = [1,2,3,4,5]
random.sample(a,len(a))

[3, 4, 5, 2, 1]

In [142]:
sum([[1,2,3],[4,5,6]],[])

[1, 2, 3, 4, 5, 6]

In [174]:
a = [{'id': 58.0, 'x': 15.0, 'y': 10.0, 'd': 10.0, 'S': 20.0, 'f': 1.0, 'a': 1.0, 'list': 1.0, 'O': 51.0, 'C': 182.0}, {'id': 77.0, 'x': 23.0, 'y': 3.0, 'd': 10.0, 'S': 7.0, 'f': 1.0, 'a': 1.0, 'list': 1.0, 'O': 49.0, 'C': 180.0}, {'id': 28.0, 'x': 92.0, 'y': 30.0, 'd': 10.0, 'S': 10.0, 'f': 1.0, 'a': 1.0, 'list': 1.0, 'O': 55.0, 'C': 174.0}, {'id': 32.0, 'x': 87.0, 'y': 30.0, 'd': 10.0, 'S': 10.0, 'f': 1.0, 'a': 1.0, 'list': 1.0, 'O': 61.0, 'C': 178.0}]

In [175]:
sum(a,[])

TypeError: can only concatenate list (not "dict") to list